In [140]:
from resolver import OccurrenceResolver

In [1]:
import requests
from xml.etree import ElementTree

In [2]:
services = {'wa' : {
                'endpoint': 'http://geossdi.dmp.wa.gov.au/services/wfs',
                'feature_type' : 'mo:MinOccView',
                'er_version' : 1,
                'lite_version': 1,
            }, 'nt': {
                'endpoint' : 'http://geology.data.nt.gov.au:80/geoserver/wfs',
                'feature_type' : 'mo:MinOccView',
                'er_version' : 1,
                'lite_version': 1,
            },  'vic': {
                'endpoint': 'http://geology.data.vic.gov.au/nvcl/wfs',
                'feature_type' : 'mo:MinOccView',
                'er_version' : 1,
                'lite_version': 1,
            },  'sa': {
                'endpoint': 'https://sarigdata.pir.sa.gov.au/nvcl/geoserver/wfs',
                'feature_type' : 'erl:MineralOccurrenceView',
                'lite_version': 2,
                'er_version' : 1
            },  'nsw': {
                'endpoint': 'https://gs.geoscience.nsw.gov.au/geoserver/wfs',
                'feature_type' : 'mo:MinOccView',
                'lite_version': 1,
                'er_version' : 1
            },  'tas': {
                'endpoint': 'http://www.mrt.tas.gov.au:80/web-services/wfs',
                'feature_type' : 'erl:MineralOccurrenceView',
                'lite_version': 1,
                'er_version' : 2
            }
           }

In [3]:
feature_types = {'mo:MinOccView' : {
            'name' : 'mo:name',
            'shape' : 'mo:shape/gml:Point/gml:pos',
            'spec_uri' : 'mo:earthResourceSpecification_uri'
        }, 'erl:MineralOccurrenceView' : {
            'name' : 'erl:name',
            'shape' : 'erl:shape/gml:Point/gml:pos',
            'spec_uri' : 'erl:specification_uri'
        }}

In [4]:
ns = {'wfs': 'http://www.opengis.net/wfs',
      'er': 'urn:cgi:xmlns:GGIC:EarthResource:1.1',
     'gml':'http://www.opengis.net/gml',
     'mo': 'http://xmlns.geoscience.gov.au/minoccml/1.0',
     'gsml': 'urn:cgi:xmlns:CGI:GeoSciML:2.0'}

In [10]:
for state, service in services.items():
    print(state)
    process_endpoints(state, service)

wa
http://geossdi.dmp.wa.gov.au/resource/feature/gswa/mineraloccurrence/S0000001
S0000001 Pieces of Eight - Admiral Hill -28.565092 122.480745 [('Au', 1, None), ('Ag', 2, None)]
http://geossdi.dmp.wa.gov.au/resource/feature/gswa/mineraloccurrence/S0000002
S0000002 Albury Heath -26.78737 118.574407 [('Au', 1, None)]
http://geossdi.dmp.wa.gov.au/resource/feature/gswa/mineraloccurrence/S0000005
S0000005 Aspacia -29.69063 121.012802 [('Au', 1, None)]
nt
http://geology.data.nt.gov.au/resource/feature/ntgs/mineraloccurrence/1
1 er.mineraloccurrence.1 -15.1666737321045 129.090687555216 [('Coal', None, 'major')]
http://geology.data.nt.gov.au/resource/feature/ntgs/mineraloccurrence/10
10 er.mineraloccurrence.10 -17.2339668532906 131.255469431007 [('Qtza', None, 'major')]
http://geology.data.nt.gov.au/resource/feature/ntgs/mineraloccurrence/100
100 er.mineraloccurrence.100 -12.3601135984816 132.943601783326 [('U', None, 'major')]
vic
http://geology.data.vic.gov.au/resource/feature/GSV/mineralocc

In [71]:
def process_endpoints(state, service):
    endpoint = service['endpoint']
    feature_type = service['feature_type']

    parameters = {'service': 'WFS',
              'version' : '1.1.0',
              'request' : 'GetFeature',
              'typeName' : feature_type,
              'maxFeatures' : 3}
    if service['lite_version'] == 1:
        ns['erl'] = 'http://xmlns.earthresourceml.org/earthresourceml-lite/1.0'
        ns['wfs'] = 'http://www.opengis.net/wfs'
        ns['gml'] = 'http://www.opengis.net/gml'
        occurrences_path = "gml:featureMembers/" + feature_type
    else:
        ns['erl'] = 'http://xmlns.earthresourceml.org/earthresourceml-lite/2.0'
        ns['wfs'] = 'http://www.opengis.net/wfs/2.0'
        ns['gmdef extract_commodities(specification_uri, endpoint):
    feature_id = 'er.'+'.'.join(url.split('/')[-2:])
    commods = []
    response = requests.get(specification_uri)
    print(response)
    root = ElementTree.fromstring(response.text)
    commodities = root.findall('gml:featureMembers/er:MineralOccurrence/er:commodityDescription/er:Commodity',ns)
    order = 1
    for commodity in commodities:
        
        code = commodity.find('er:commodityName',ns).text.replace('urn:cgi:classifier:GA:commodity:','')
        order = commodity.find('er:commodityRank',ns)
        importance = commodity.find('er:commodityImportance',ns)
        if order is not None:
            order = int(order.text)
        if importance is not None:
            importance = importance.text
        commods.append((code,order,importance))
    return commodsl'] = 'http://www.opengis.net/gml/3.2'
        parameters['outputFormat'] = 'gml32'
        occurrences_path = "wfs:member/" + feature_type
    response=requests.get(endpoint,params=parameters)
    
    print(response.url)
    root = ElementTree.fromstring(response.text)
    occurrences_path = "gml:featureMembers/" + feature_type
    features = root.findall('gml:featureMembers',ns) or root.findall('gml:featureMember',ns) or root.findall('wfs:member',ns)
    process_features(features, feature_type, endpoint)
    

In [72]:
def process_features(features, feature_type, endpoint):
    for feature in features:
        occurrences = feature.findall(feature_type, ns)
        process_occurrences(occurrences, feature_type, endpoint)

In [87]:
def process_occurrences(occurrences, feature_type, endpoint):
    for occurrence in occurrences:
        name_path = feature_types[feature_type]['name']
        spec_path = feature_types[feature_type]['spec_uri']
        shape_path = feature_types[feature_type]['shape']
        #gml_id = occurrence.attrib['{http://www.opengis.net/gml}id']
        otherid = None #gml_id.split('.')[-1]
        name = occurrence.find(name_path,ns).text
        latlng = occurrence.find(shape_path,ns)
        specification_uri = occurrence.find(spec_path, ns).text
        latitude, longitude = [float(pos) for pos in latlng.text.split(' ')]
        print(latlng)
        print(specification_uri)
        commods = extract_commodities(specification_uri, endpoint)
        print(otherid, name, latitude, longitude, commods)

In [88]:
def extract_commodities(specification_uri, endpoint):
    feature_id = 'er.'+'.'.join(url.split('/')[-2:])
    commods = []
    response = requests.get(specification_uri)
    print(response)
    root = ElementTree.fromstring(response.text)
    commodities = root.findall('gml:featureMembers/er:MineralOccurrence/er:commodityDescription/er:Commodity',ns)
    order = 1
    for commodity in commodities:
        
        code = commodity.find('er:commodityName',ns).text.replace('urn:cgi:classifier:GA:commodity:','')
        order = commodity.find('er:commodityRank',ns)
        importance = commodity.find('er:commodityImportance',ns)
        if order is not None:
            order = int(order.text)
        if importance is not None:
            importance = importance.text
        commods.append((code,order,importance))
    return commods


In [136]:
def get_feature(url, feature_id):
    parameters = {'service': 'WFS',
              'version' : '1.1.0',
              'request' : 'GetFeature',
              'typeName' : 'er:MineralOccurrence',
              'featureId' : feature_id}
    response = requests.get(url, params=parameters)
    print(response.text)

In [142]:
OccurrenceResolver.get_feature('https://gs.geoscience.nsw.gov.au/geoserver/wfs','er.mineraloccurrence.101000')

NameError: name 'url' is not defined

In [137]:
def get_order(commodity):
    rank_path = 'er:commodityRank'
    importance_path 

In [90]:
process_endpoints('nsw', services['nsw'])

https://gs.geoscience.nsw.gov.au/geoserver/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=mo%3AMinOccView&maxFeatures=3
<Element '{http://www.opengis.net/gml}pos' at 0x7f589e46fcc8>
http://gs.geoscience.nsw.gov.au/resource/feature/gsnsw/mineraloccurrence/101000
<Response [404]>


ParseError: syntax error: line 1, column 49 (<string>)

In [85]:
url='https://sarigdata.pir.sa.gov.au/nvcl/geoserver/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=erl%3AMineralOccurrenceView&maxFeatures=3&outputFormat=gml32'

In [111]:
headers = {'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [114]:
session = requests.Session()

response = requests.get('http://gs.geoscience.nsw.gov.au/resource/feature/gsnsw/mineraloccurrence/101000', headers=headers)

In [115]:
response.text

'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>404 Not Found</title>\n</head><body>\n<h1>Not Found</h1>\n<p>The requested URL /resource/feature/gsnsw/mineraloccurrence/101000 was not found on this server.</p>\n<hr>\n<address>Apache/2.4.18 (Ubuntu) Server at gs.geoscience.nsw.gov.au Port 80</address>\n</body></html>\n'

In [116]:
url = 'http://gs.geoscience.nsw.gov.au/resource/feature/gsnsw/mineraloccurrence/101000'

'er.mineraloccurrence.101000'